<a href="https://colab.research.google.com/github/wurDevTim/Workshop_P4P/blob/main/correcting_meassurement_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Correcting for meassurement time
workshop part 2


## setup
Next to python, google colab can also be used to run R notebooks like this one.
To switch:  open the menu 'Runtime' at the top, select 'change runtime type'. And select R at runtime type.

In [1]:
# Install packages
install.packages("LMMsolver")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘dotCall64’, ‘agridat’, ‘maps’, ‘Rcpp’, ‘sp’, ‘spam’




In [3]:
# Import
library(LMMsolver)

In [4]:
# Function which uses the LMM solver to compute the spline.
compute_spline <- function(df, plant_identifier, trait_list)
{
  ### Fit 1D spline per plant
  for (i in c(1:length(unique(df[[plant_identifier]])))){
    plant_id = unique(df[[plant_identifier]])[i]
    one_plant <- df[df[[plant_identifier]] == plant_id,]
    datenum = one_plant[['datenum']]
    preddates <- data.frame(datenum = min(one_plant$date):max(one_plant$date))
    # Each day has 24*60*60 = 86400 hours
    preddates <- preddates * 86400

    # Fit 1D spline per trait
    for (trait in trait_list){
      # Nan values will be removed, but they do cause warnings.
      m1 <- LMMsolve(fixed = as.formula(paste(trait, "~", 1)),
                     spline = ~spl1D(x = datenum, nseg = 20),
                     data = one_plant)
      summary(m1)
      prediction <- obtainSmoothTrend(m1, newdata = preddates,
                              includeIntercept = T)
      # Rename ypred column
      names(prediction)[names(prediction) == 'ypred'] <- trait

      # Combine results
      if (trait == trait_list[1]){
        plant_predictions <- prediction
      } else {
        plant_predictions <- merge(plant_predictions, prediction, by='datenum')
      }
    }
    plant_predictions[[plant_identifier]] = plant_id
    if (i == 1){
      all_predictions <- plant_predictions
    } else {

      all_predictions <- rbind(all_predictions, plant_predictions)
    }
  }
  return(all_predictions)
}

# Example
In this example the cropreporter data from Lucia is used, which has been analysed beforehand.

Note: Systems like the cropreporter in NPEC use the local time. If your unlucky your experiment contains both winter & summer time. In this case we would advice to switch to UTC.

In [ ]:
# Load the data
df <- read.csv('./data/94_pr_cropreporter_traits_table.csv', sep = ";")